# Agentic RAG Pipeline Demo

This notebook demonstrates a complete RAG pipeline for enterprise document analysis.

In [1]:
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv(project_root / ".env")

True

In [2]:
%run ../scripts/generate_sample_pdfs.py

Created: /Users/rohiogula/rohi/learn/agentic-rag/data/pdfs/technical_report.pdf
Created: /Users/rohiogula/rohi/learn/agentic-rag/data/pdfs/product_spec.pdf
Created: /Users/rohiogula/rohi/learn/agentic-rag/data/pdfs/financial_summary.pdf

All sample PDFs created in: /Users/rohiogula/rohi/learn/agentic-rag/data/pdfs


/Users/rohiogula/rohi/learn/agentic-rag/scripts/generate_sample_pdfs.py:16: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "Industrial Equipment Technical Report", ln=True, align="C")
/Users/rohiogula/rohi/learn/agentic-rag/scripts/generate_sample_pdfs.py:20: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "1. Executive Summary", ln=True)
/Users/rohiogula/rohi/learn/agentic-rag/scripts/generate_sample_pdfs.py:35: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "2. Operating Specifications", ln=True)
/Users/rohiogula/rohi/learn/agentic-rag/scripts/generate_sample_pdfs.py:54: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.


## Phase 1: Document Ingestion

In [3]:
from agentic_rag.ingestion.pdf_parser import PDFParser

parser = PDFParser(extract_images=True, image_dpi=150)
pdf_dir = Path.cwd().parent / "data" / "pdfs"
documents = parser.parse_directory(pdf_dir)

print(f"Parsed {len(documents)} documents:")
for doc in documents:
    print(f"  - {doc.filename}: {doc.total_pages} pages")

Parsed 3 documents:
  - financial_summary.pdf: 1 pages
  - technical_report.pdf: 2 pages
  - product_spec.pdf: 1 pages


## Phase 2: Table Extraction

In [4]:
from openai import OpenAI
from agentic_rag.ingestion.table_extractor import TableExtractor

client = OpenAI()
table_extractor = TableExtractor(client=client, model="gpt-4o")

all_tables = []
for doc in documents:
    tables = table_extractor.extract_from_pages(doc.pages, only_table_pages=True)
    all_tables.extend(tables)
    print(f"{doc.filename}: Extracted {len(tables)} tables")

financial_summary.pdf: Extracted 2 tables
technical_report.pdf: Extracted 1 tables
product_spec.pdf: Extracted 2 tables


## Phase 3: Chunking

In [5]:
from agentic_rag.indexing.chunker import DocumentChunker, ChunkType

chunker = DocumentChunker(chunk_size=512, chunk_overlap=50)
all_chunks = []

for doc in documents:
    for page in doc.pages:
        chunks = chunker.chunk_text(text=page.text, source_file=doc.filename, page_num=page.page_num)
        all_chunks.extend(chunks)

for i, table in enumerate(all_tables):
    chunk = chunker.chunk_table(table_json=table.table_json, table_summary=table.table_summary,
                                source_file=table.source_file, page_num=table.page_num, chunk_id=f"table_{i}")
    all_chunks.append(chunk)

print(f"Total chunks: {len(all_chunks)}")

Total chunks: 9


## Phase 4: Indexing

In [6]:
from agentic_rag.indexing.hybrid_index import HybridIndex

index = HybridIndex(collection_name="demo_docs", openai_client=client)
index.clear()
added = index.add_chunks(all_chunks)
print(f"Added {added} chunks to index")

Added 9 chunks to index


## Phase 5: Retrieval

In [7]:
from agentic_rag.retrieval.hybrid_retriever import HybridRetriever

retriever = HybridRetriever(index=index, openai_client=client, explain_retrievals=False)

query = "What is the maximum operating temperature?"
result = retriever.retrieve(query, n_results=3)
print(f"Query: {query}")
for chunk in result.chunks:
    print(f"  - {chunk.source_file}: {chunk.relevance_score:.2f}")

Query: What is the maximum operating temperature?
  - technical_report.pdf: 0.51
  - technical_report.pdf: 0.44
  - technical_report.pdf: 0.43


## Phase 6: Reasoning Agent

In [8]:
from agentic_rag.agent.guardrails import create_manufacturing_guardrail
from agentic_rag.agent.reasoning import ReasoningAgent

guardrail = create_manufacturing_guardrail()
agent = ReasoningAgent(retriever=retriever, guardrail=guardrail, openai_client=client, model="gpt-4o")

In [9]:
query = "What is the current operating temperature and is it safe?"
result = agent.reason(query, n_chunks=5)

print(f"Query: {query}")
print(f"Answer: {result.answer}")
print(f"Confidence: {result.confidence:.0%}")
print(f"Risk Flags: {result.risk_flags}")

Query: What is the current operating temperature and is it safe?
Answer: The current operating temperature is 72°C, which is approaching the critical limit of 80°C. It is not entirely safe as it is flagged with a WARNING status, indicating a high risk of exceeding the safe operating parameters if not monitored closely.
Confidence: 90%
Risk Flags: []


## Phase 7: Structured Output

In [10]:
from agentic_rag.output.synthesizer import OutputSynthesizer

synthesizer = OutputSynthesizer(openai_client=client)
output = synthesizer.synthesize(query=query, reasoning_output=result)
print(synthesizer.format_for_display(output))

ANALYSIS REPORT

Query: What is the current operating temperature and is it safe?

SUMMARY
----------------------------------------
The current operating temperature is 72°C, which is nearing the critical threshold of 80°C. This situation is flagged with a WARNING status, indicating a significant risk of exceeding safe operating parameters if not closely monitored.

KEY FINDINGS
----------------------------------------
  1. The operating temperature is currently at 72°C.
  2. The critical limit for safe operation is 80°C.
  3. The system is under a WARNING status due to the high risk of exceeding safe temperature limits.
  4. Immediate monitoring and potential intervention are required to prevent overheating.
  5. No additional risk flags were identified at this time.

EXTRACTED DATA
----------------------------------------
  temperature: 72 °C

CITATIONS
----------------------------------------
  [1] technical_report.pdf, Page 1

Confidence: 90%
Generated: 2026-01-21T11:55:23.220086


In [11]:
print(output.to_json())

{
  "query": "What is the current operating temperature and is it safe?",
  "summary": "The current operating temperature is 72\u00b0C, which is nearing the critical threshold of 80\u00b0C. This situation is flagged with a WARNING status, indicating a significant risk of exceeding safe operating parameters if not closely monitored.",
  "key_findings": [
    "The operating temperature is currently at 72\u00b0C.",
    "The critical limit for safe operation is 80\u00b0C.",
    "The system is under a WARNING status due to the high risk of exceeding safe temperature limits.",
    "Immediate monitoring and potential intervention are required to prevent overheating.",
    "No additional risk flags were identified at this time."
  ],
  "extracted_data": {
    "temperature": {
      "value": "72",
      "unit": "\u00b0C",
      "source": 1
    }
  },
  "risk_flags": [],
  "citations": [
    {
      "id": 1,
      "source_file": "technical_report.pdf",
      "page": 1,
      "chunk_type": "text"

## Phase 8: Evaluation Examples

Quick sanity checks across different question types.


In [ ]:
eval_queries = [
    "What is the maximum operating temperature?",
    "List the revenue figures and any risk flags.",
    "Summarize the key requirements in the product spec.",
]

for q in eval_queries:
    res = agent.reason(q, n_chunks=5)
    print("\nQuery:", q)
    print("Answer:", res.answer)
    print("Confidence:", f"{res.confidence:.0%}")
    print("Citations:", [c.source_file for c in res.citations])
